In [8]:
import json
import pandas as pd
import tweepy
import json
import pandas as pd
import re
import nltk
from textblob import TextBlob


api_key = "aJTgi4d1H1zmQNkQuHeualNhP"
api_secrets = "yPFdp2Bbib25r1iPQCqBuPq8B9UzoJbIdpv1jgzEZFEg1eJl6X"
access_token = "1409382627124019204-oPpzVGuCwyFfQTfoocFhgHi68whhog"
access_secret = "esgwpH5gnRfNB0SpiHkO52mZSI5VKKnn8SG2pafEnzj2b"
 
# Authenticate to Twitter
auth = tweepy.OAuthHandler(api_key,api_secrets)
auth.set_access_token(access_token,access_secret)
 
api = tweepy.API(auth)

def get_tweets_from_api(ids):
    for i in range(0, len(ids), 100):
        split = ids[i:i+100]
        for tweet in api.lookup_statuses(split, tweet_mode='extended'):
            tweets_dict[tweet.id_str] = {}
            tweets_dict[tweet.id_str]['text'] = tweet.full_text
            tweets_dict[tweet.id_str]['user'] = tweet.user.screen_name
            tweets_dict[tweet.id_str]['verified'] = tweet.user.verified
            tweets_dict[tweet.id_str]['followers'] = tweet.user.followers_count
            tweets_dict[tweet.id_str]['num_statuses'] = tweet.user.statuses_count
            tweets_dict[tweet.id_str]['likes'] = tweet.favorite_count
            tweets_dict[tweet.id_str]['retweets'] = tweet.retweet_count
            hashtag = tweet.entities['hashtags']
            if hashtag == []:
                tweets_dict[tweet.id_str]['hashtag'] = ""
            else:
                tweets_dict[tweet.id_str]['hashtag'] = hashtag[0]['text']

                    
source_ids = []
file = open("covid.data.txt")
threads = file.readlines()
file.close()
source_ids = [thread.replace('\n', '').split(',')[0] for thread in threads]

tweets_dict = {}

get_tweets_from_api(source_ids)
                      
tweet_text = []
tweet_user = []
tweet_verified = []
tweet_likes = []
tweet_retweets = []
tweet_hashtag = []
tweet_follow = []
tweet_statuses = []

for tweet_id in tweets_dict.keys():
    tweet_text.append(tweets_dict[tweet_id]['text'])
    tweet_user.append(tweets_dict[tweet_id]['user'])
    tweet_verified.append(tweets_dict[tweet_id]['verified'])
    tweet_likes.append(tweets_dict[tweet_id]['likes'])
    tweet_retweets.append(tweets_dict[tweet_id]['retweets'])
    tweet_hashtag.append(tweets_dict[tweet_id]['hashtag'])
    tweet_follow.append(tweets_dict[tweet_id]['followers'])
    tweet_statuses.append(tweets_dict[tweet_id]['num_statuses'])
    
covid_ids_df = pd.DataFrame({'id': tweets_dict.keys(), 
                             'text': tweet_text, 
                             'user': tweet_user,
                             'verified': tweet_verified,
                             'likes': tweet_likes,
                             'retweets': tweet_retweets,
                             'hashtag': tweet_hashtag,
                             'followers': tweet_follow,
                             'statuses': tweet_statuses},
                              index=tweets_dict.keys())

covid_ids_df.to_csv('covid_id_df.csv', header=True)

In [1]:
import json
import pandas as pd
import tweepy
import json
import pandas as pd
import re
import nltk
from textblob import TextBlob

source_ids = []
file = open("covid.data.txt")
threads = file.readlines()
file.close()
source_ids = [thread.replace('\n', '').split(',')[0] for thread in threads]

covid_ids_df = pd.read_csv('covid_id_df.csv', index_col=0) 
covid_ids_df.drop('id',axis=1,inplace=True)
covid_ids_df.head()

,text,user,verified,likes,retweets,hashtag,followers,statuses
1250320429564063744,अहमदाबाद के सरकारी अस्पताल में तो धर्म के हिस...,ajitanjum,False,4480,1826,NaN,1395584,32440
1275965523625418753,Trump has royally fucked up this country.,StephenKing,True,239253,32010,NaN,6672631,6917
1278131205754388480,Governor says people know what to do to keep C...,wsbtv,True,123,34,NaN,1201691,621441
1262509072051470340,Earlier today the NYPD shut down a Yeshiva con...,NYCMayor,True,3800,954,NaN,1598092,27474
1278159404324405248,Major League Soccer COVID-19 Testing Update - ...,MLS_PR,True,52,26,NaN,7793,1760


In [2]:
covid_labels = pd.read_csv('covid_predictions.csv', index_col=0)
covid_labels_df = pd.DataFrame({'label': list(covid_labels['Predicted'])}, index=source_ids)
covid_labels_df.head()

,label
1272262651100434433,0
1287153210990395392,0
1266555444283179008,0
1257715199655755779,0
1274338812173393920,1


In [3]:
covid_ids_df.index= covid_ids_df.index.map(str)
covid_df_full = covid_labels_df.join(covid_ids_df)
cols = list(covid_df_full.columns)[1:] + [list(covid_df_full.columns)[0]]
covid_df_full = covid_df_full.reindex(columns=cols, fill_value=0)
covid_df_full.fillna(value="",inplace=True)
covid_df_full

,text,user,verified,likes,retweets,hashtag,followers,statuses,label
1272262651100434433,"According to the New York Times, Warner Bros. ...",TheFilmUpdates,False,260,25,Tenet,45008,5654,0
1287153210990395392,Hurricane Hanna has made landfall in Texas.\n\...,TexasTribune,True,580,479,,297392,110931,0
1266555444283179008,Monkeys on the loose in India with stolen coro...,thehill,True,164,72,,4.36246e+06,1.0139e+06,0
1257715199655755779,,,,,,,,,0
1274338812173393920,"“If Trump felt comfortable having it here, the...",HeidiNBC,True,1952,786,,204299,19775,1
...,...,...,...,...,...,...,...,...,...
1249502859185590272,I wonder how many lives could’ve been saved if...,funder,True,10632,2567,,1.0736e+06,305211,1
1284050414619459586,The @thetimes front page on 17th March. The fi...,NadineDorries,True,772,227,,128557,29621,0
1274505289614725122,Trump just completed the racism trifecta in a ...,DNCWarRoom,True,915,367,,88433,10988,1
1267884642637676545,Here are a few of my photographs from today’s ...,Jess__Taylor__,False,343,153,COVID19,8044,692,0


In [4]:
def sentiment_df(df):
    preprocessed = df
    text = list(preprocessed['text'])
    sentiment = []
    for i in range(len(text)):
        text[i] = " ".join(filter(lambda x:x[0]!='@', text[i].split()))
        text[i] = " ".join(filter(lambda x:x[0:4]!='http', text[i].split()))
        text[i] = " ".join(filter(lambda x:x[0:4]!='&amp', text[i].split()))
        text[i] = re.sub(r'[^a-zA-Z ]','',text[i])
        text[i] = " ".join(filter(lambda x:x[0:1]!='Q', text[i].split()))
        text[i] = " ".join(filter(lambda x:x[0:1]!='A', text[i].split()))
        sentiment.append(TextBlob(text[i]).sentiment.polarity)
    return sentiment

sentiment = sentiment_df(covid_df_full)
covid_df_full.insert(6,'sentiment',sentiment)
covid_df_full

,text,user,verified,likes,retweets,hashtag,sentiment,followers,statuses,label
1272262651100434433,"According to the New York Times, Warner Bros. ...",TheFilmUpdates,False,260,25,Tenet,0.296591,45008,5654,0
1287153210990395392,Hurricane Hanna has made landfall in Texas.\n\...,TexasTribune,True,580,479,,0.000000,297392,110931,0
1266555444283179008,Monkeys on the loose in India with stolen coro...,thehill,True,164,72,,-0.076923,4.36246e+06,1.0139e+06,0
1257715199655755779,,,,,,,0.000000,,,0
1274338812173393920,"“If Trump felt comfortable having it here, the...",HeidiNBC,True,1952,786,,0.400000,204299,19775,1
...,...,...,...,...,...,...,...,...,...,...
1249502859185590272,I wonder how many lives could’ve been saved if...,funder,True,10632,2567,,0.200000,1.0736e+06,305211,1
1284050414619459586,The @thetimes front page on 17th March. The fi...,NadineDorries,True,772,227,,0.125000,128557,29621,0
1274505289614725122,Trump just completed the racism trifecta in a ...,DNCWarRoom,True,915,367,,-0.388889,88433,10988,1
1267884642637676545,Here are a few of my photographs from today’s ...,Jess__Taylor__,False,343,153,COVID19,-0.200000,8044,692,0


In [5]:
covid_df_full.drop(covid_df_full[covid_df_full.text == ""].index, inplace=True)
covid_df_full

,text,user,verified,likes,retweets,hashtag,sentiment,followers,statuses,label
1272262651100434433,"According to the New York Times, Warner Bros. ...",TheFilmUpdates,False,260,25,Tenet,0.296591,45008,5654,0
1287153210990395392,Hurricane Hanna has made landfall in Texas.\n\...,TexasTribune,True,580,479,,0.000000,297392,110931,0
1266555444283179008,Monkeys on the loose in India with stolen coro...,thehill,True,164,72,,-0.076923,4.36246e+06,1.0139e+06,0
1274338812173393920,"“If Trump felt comfortable having it here, the...",HeidiNBC,True,1952,786,,0.400000,204299,19775,1
1278703879828303877,DISTURBING: Alabama officials say some student...,ABC7,True,471,256,COVID19,-0.125000,1.17887e+06,256495,0
...,...,...,...,...,...,...,...,...,...,...
1249502859185590272,I wonder how many lives could’ve been saved if...,funder,True,10632,2567,,0.200000,1.0736e+06,305211,1
1284050414619459586,The @thetimes front page on 17th March. The fi...,NadineDorries,True,772,227,,0.125000,128557,29621,0
1274505289614725122,Trump just completed the racism trifecta in a ...,DNCWarRoom,True,915,367,,-0.388889,88433,10988,1
1267884642637676545,Here are a few of my photographs from today’s ...,Jess__Taylor__,False,343,153,COVID19,-0.200000,8044,692,0


In [6]:
covid_df_full.to_csv('covid_df.csv', header=True)

In [7]:
file = open("covid.data.txt")
threads = file.readlines()
file.close()

labels = pd.read_csv("covid.label.txt")

all_ids = [thread.replace('\n', '').split(',') for thread in threads]

reply_source = [] 
reply_ids = []

for thread in all_ids:
    if len(thread) > 1:
        for i in range(len(thread) - 1):
            reply_source.append(thread[0])
            reply_ids.append(thread[i+1])

reply_source

['1287153210990395392',
 '1257715199655755779',
 '1257715199655755779',
 '1257715199655755779',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1274338812173393920',
 '1249421782630772746',
 '1249421782630772746',
 '1249421782630772746',
 '1249421782630772746',
 '1249421782630772746',
 '1249421782630772746',
 '1274091468890632192',
 '1283500526450159616',
 '1283500526450159616',
 '1283500526450159616',
 '1283500526450159616',
 '12835005264501

In [8]:
class_dict = {}

source_ids = [thread.replace('\n', '').split(',')[0] for thread in threads]
label = list(labels['Predicted'])

for i in range(len(source_ids)):
    class_dict[source_ids[i]] = label[i]

class_dict

{'1272262651100434433': 0,
 '1287153210990395392': 0,
 '1266555444283179008': 0,
 '1257715199655755779': 0,
 '1274338812173393920': 0,
 '1278703879828303877': 0,
 '1249421782630772746': 0,
 '1274091468890632192': 0,
 '1278159404324405248': 0,
 '1283500526450159616': 0,
 '1275627556289511424': 0,
 '1257250378514526208': 0,
 '1265729173961748480': 0,
 '1265536236191731715': 0,
 '1252660417341485060': 0,
 '1239213982243868672': 1,
 '1262904939086516225': 1,
 '1242502830918979585': 0,
 '1275522992672509952': 1,
 '1264422394732527616': 0,
 '1258433414291734529': 0,
 '1275965523625418753': 1,
 '1247303589501992962': 0,
 '1280522599869579264': 0,
 '1245128889116102658': 1,
 '1242954004914524160': 0,
 '1274036638042198017': 0,
 '1277981235403522048': 0,
 '1269281877246062598': 1,
 '1260615891701706752': 0,
 '1254653822225518592': 0,
 '1251608760793083905': 0,
 '1274126142119837696': 0,
 '1244314233669668866': 0,
 '1245863874789642240': 1,
 '1245372706633236484': 0,
 '1252361062575296515': 0,
 

In [9]:
reply_class = []

for x in reply_source:
    reply_class.append(class_dict[x])
    
reply_class

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [10]:
replies_df = pd.DataFrame({'reply_id': reply_ids,
                           'source_class': reply_class})

replies_df

,reply_id,source_class
0,1287191952115605505,0
1,1258212704961155073,0
2,1257843417503141895,0
3,1258230330210779143,0
4,1274369294558801921,0
...,...,...
237218,1265936765988286465,1
237219,1265828881182928901,1
237220,1265849632598433795,1
237221,1265825373868228608,1
